In [1]:
import pathlib
import torch
from absl import app
from absl import flags
import esm
from esm import pretrained, MSATransformer, FastaBatchedDataset
from tqdm import tqdm
from Bio import PDB
from typing import List

In [2]:
from rdkit import Chem
path_protein = "../../dataset2016/1a4k/1a4k_protein.pdb"
path_pocket = "../../dataset2016/1a4k/1a4k_pocket.pdb"

## Get sequence from protein

In [5]:
pdb_parser = PDB.PDBParser(QUIET=True)
structure = pdb_parser.get_structure("1a4k", path_protein)
ppb = PDB.PPBuilder()
peptides = ppb.build_peptides(structure)
polypeptide_sequences = [pp.get_sequence() for pp in peptides]

In [6]:
first_seq = polypeptide_sequences[0]
element = first_seq[0]
print(element, polypeptide_sequences[0])

E ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG


In [7]:
PDB.Polypeptide.index_to_three(1)

'CYS'

In [17]:
a = str(polypeptide_sequences[0])

In [16]:
b = str(polypeptide_sequences[2])

In [11]:
from nltk.metrics.distance import *

In [18]:
edit_distance(a, b)

1

In [19]:
l = []
for i in polypeptide_sequences[2]:
    l.append(PDB.Polypeptide.one_to_three(i))
l

/home/sofiaguerreiro/molecules-binding/.env/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:161: BiopythonDeprecationWarning: 'one_to_three' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_1to3'.
  warnings.warn(


['GLU',
 'LEU',
 'VAL',
 'MET',
 'THR',
 'GLN',
 'THR',
 'PRO',
 'LEU',
 'SER',
 'LEU',
 'PRO',
 'VAL',
 'SER',
 'LEU',
 'GLY',
 'ASP',
 'GLN',
 'ALA',
 'SER',
 'ILE',
 'SER',
 'CYS',
 'ARG',
 'SER',
 'SER',
 'GLN',
 'SER',
 'LEU',
 'LEU',
 'HIS',
 'SER',
 'ASN',
 'GLY',
 'ASN',
 'THR',
 'TYR',
 'LEU',
 'HIS',
 'TRP',
 'TYR',
 'LEU',
 'GLN',
 'LYS',
 'PRO',
 'GLY',
 'GLN',
 'SER',
 'PRO',
 'LYS',
 'LEU',
 'LEU',
 'ILE',
 'TYR',
 'LYS',
 'VAL',
 'SER',
 'ASN',
 'ARG',
 'PHE',
 'SER',
 'GLY',
 'VAL',
 'PRO',
 'ASP',
 'ARG',
 'PHE',
 'SER',
 'GLY',
 'SER',
 'GLY',
 'SER',
 'GLY',
 'THR',
 'ASP',
 'PHE',
 'THR',
 'LEU',
 'LYS',
 'ILE',
 'SER',
 'ARG',
 'VAL',
 'GLU',
 'ALA',
 'GLU',
 'ASP',
 'LEU',
 'GLY',
 'VAL',
 'TYR',
 'PHE',
 'CYS',
 'SER',
 'GLN',
 'VAL',
 'THR',
 'HIS',
 'VAL',
 'PRO',
 'PRO',
 'THR',
 'PHE',
 'GLY',
 'GLY',
 'GLY',
 'THR',
 'LYS',
 'LEU',
 'GLU',
 'ILE',
 'LYS',
 'ARG',
 'THR',
 'VAL',
 'ALA',
 'ALA',
 'PRO',
 'SER',
 'VAL',
 'PHE',
 'ILE',
 'PHE',
 'PRO',
 'PRO',


In [20]:
for i in polypeptide_sequences:
    print(len(i))

217
217
216
219


In [ ]:
molecule = Chem.MolFromPDBFile(path_protein)


### Get residues numbers and residues ids

In [3]:
from Bio import PDB
parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("1a4k", path_protein)

In [19]:
model = structure[0]
chain = model['A']
residues = chain.get_residues()
for residue in residues:
    if residue.get_resname() == "SER":
        print(residue.get_resname(), residue.get_id()[1], residue.get_full_id(), residue.get_atoms())
    # print(residue.get_resname(), residue.get_id()[1], residue.get_full_id(), residue.get_atoms())

SER 10 ('1a4k', 0, 'A', (' ', 10, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 14 ('1a4k', 0, 'A', (' ', 14, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 20 ('1a4k', 0, 'A', (' ', 20, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 22 ('1a4k', 0, 'A', (' ', 22, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 25 ('1a4k', 0, 'A', (' ', 25, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 26 ('1a4k', 0, 'A', (' ', 26, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 1027 ('1a4k', 0, 'A', (' ', 1027, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 5027 ('1a4k', 0, 'A', (' ', 5027, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 43 ('1a4k', 0, 'A', (' ', 43, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 52 ('1a4k', 0, 'A', (' ', 52, ' ')) <generator object Residue.get_atoms at 0x7fb12e7ff760>
SER 56 ('1a4k', 0, 'A', (' ', 56, ' ')) <g

### For one atom find the corresponding residue

In [3]:
def find_correct_element_of_chain(residue_name, residue_id, atom_symbol, atom_coords, structure):
    for model in structure:
        for i, chain in enumerate(model):
            for j,residue in enumerate(chain):
                if residue.get_resname() == residue_name and residue.get_id()[1] == residue_id:
                    for atom in residue:
                        if atom.element == atom_symbol:
                            if (atom.get_coord() - atom_coords < 0.1).all():
                                return i,j
    return None, None

In [114]:
find_correct_element_of_chain("HIS", 4027, "C", coords[1], structure)

[35.6340011  48.51599884 -1.04699998]
[36.35600122 48.15100098 -2.32800011]
[34.57499918 47.45999908 -0.74699979]
[35.14299997 46.15699768 -0.28700023]
[35.58800348 45.08399963 -0.98000006]
[35.79800256 44.625       1.15400024]
[ 3.20434570e-07  0.00000000e+00 -4.57763676e-08]


(2, 30)

### For each atom in the pocket find the corresponding chains and residues

In [4]:
molecule = Chem.MolFromPDBFile(path_pocket, flavor=2, sanitize=True, removeHs=True)
conformer = molecule.GetConformer(0)
coords = conformer.GetPositions()

In [12]:
correct_chain_residue = []
for n, atom in enumerate(molecule.GetAtoms()):
    atom_coord = coords[n]
    atom_symbol = atom.GetSymbol()
    atom_res = atom.GetPDBResidueInfo()
    atom_res_name = atom_res.GetResidueName()
    atom_res_id = atom_res.GetResidueNumber()
    (i, j) = find_correct_element_of_chain(atom_res_name, atom_res_id, atom_symbol, atom_coord, structure)
    correct_chain_residue.append((i, j))
print(correct_chain_residue)

[(2, 30), (2, 30), (2, 30), (2, 30), (2, 30), (2, 30), (2, 30), (2, 30), (2, 30), (2, 30), (2, 32), (2, 32), (2, 32), (2, 32), (2, 32), (2, 32), (2, 32), (2, 32), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 36), (2, 38), (2, 38), (2, 38), (2, 38), (2, 38), (2, 38), (2, 38), (2, 38), (2, 38), (2, 38), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 40), (2, 94), (2, 94), (2, 94), (2, 94), (2, 94), (2, 94), (2, 94), (2, 94), (2, 94), (2, 95), (2, 95), (2, 95), (2, 95), (2, 95), (2, 95), (2, 95), (2, 96), (2, 96), (2, 96), (2, 96), (2, 96), (2, 96), (2, 96), (2, 97), (2, 97), (2, 97), (2, 97), (2, 97), (2, 97), (2, 97), (2, 97), (2, 97), (2, 97), (2, 98), (2, 98), (2, 98), (2, 98), (2, 98), (2, 98), (2, 98), (2, 99), (2, 99), (2, 99), (2, 99), (2, 99), (2, 99), (2, 99), (2, 100), (2, 100), (2, 100), (2, 100), (2, 100), (2, 100), (2, 100), (3, 27), (3, 27), (3, 27), (3, 27), (3

In [22]:
for i, atom in enumerate(molecule.GetAtoms()):
    chain, residue = correct_chain_residue[i]
    if len(seqs) > chain:
        res = seqs[chain][residue]
        print(i, PDB.Polypeptide.one_to_three(res), atom.GetPDBResidueInfo().GetResidueName())
    else:
        print(i, "we are in water")

0 HIS HIS
1 HIS HIS
2 HIS HIS
3 HIS HIS
4 HIS HIS
5 HIS HIS
6 HIS HIS
7 HIS HIS
8 HIS HIS
9 HIS HIS
10 ASN ASN
11 ASN ASN
12 ASN ASN
13 ASN ASN
14 ASN ASN
15 ASN ASN
16 ASN ASN
17 ASN ASN
18 TYR TYR
19 TYR TYR
20 TYR TYR
21 TYR TYR
22 TYR TYR
23 TYR TYR
24 TYR TYR
25 TYR TYR
26 TYR TYR
27 TYR TYR
28 TYR TYR
29 TYR TYR
30 HIS HIS
31 HIS HIS
32 HIS HIS
33 HIS HIS
34 HIS HIS
35 HIS HIS
36 HIS HIS
37 HIS HIS
38 HIS HIS
39 HIS HIS
40 TYR TYR
41 TYR TYR
42 TYR TYR
43 TYR TYR
44 TYR TYR
45 TYR TYR
46 TYR TYR
47 TYR TYR
48 TYR TYR
49 TYR TYR
50 TYR TYR
51 TYR TYR
52 GLN GLN
53 GLN GLN
54 GLN GLN
55 GLN GLN
56 GLN GLN
57 GLN GLN
58 GLN GLN
59 GLN GLN
60 GLN GLN
61 VAL VAL
62 VAL VAL
63 VAL VAL
64 VAL VAL
65 VAL VAL
66 VAL VAL
67 VAL VAL
68 THR THR
69 THR THR
70 THR THR
71 THR THR
72 THR THR
73 THR THR
74 THR THR
75 HIS HIS
76 HIS HIS
77 HIS HIS
78 HIS HIS
79 HIS HIS
80 HIS HIS
81 HIS HIS
82 HIS HIS
83 HIS HIS
84 HIS HIS
85 VAL VAL
86 VAL VAL
87 VAL VAL
88 VAL VAL
89 VAL VAL
90 VAL VAL
91 VAL VA

/home/sofiaguerreiro/molecules-binding/.env/lib/python3.10/site-packages/Bio/PDB/Polypeptide.py:161: BiopythonDeprecationWarning: 'one_to_three' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_1to3'.
  warnings.warn(


In [5]:
def pdb_to_sequences(pdb_filepath: str) -> List[str]:
    pdb_parser = PDB.PDBParser(QUIET=True)
    structure = pdb_parser.get_structure("X", pdb_filepath)
    # Using C-N
    ppb = PDB.PPBuilder()
    polypeptide_sequences = [
        str(pp.get_sequence()) for pp in ppb.build_peptides(structure)
    ]
    return polypeptide_sequences

In [6]:
seqs = pdb_to_sequences(path_protein)
seqs

['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG',
 'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKV',
 'ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR',
 'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP']

In [151]:
seqs[2][30]

'H'

In [155]:
PDB.Polypeptide.one_to_three(seqs[2][32])

'ASN'

In [148]:
l = []
pdb_parser = PDB.PDBParser(QUIET=True)
structure = pdb_parser.get_structure("1a4k", path_protein)
for model in structure:
    for i, chain in enumerate(model):
        ll = str()
        for j,residue in enumerate(chain):
            ll += PDB.Polypeptide.protein_letters_3to1[residue.get_resname()]
        l.append(ll)
        print(l)
print(l)

['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG']
['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG', 'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKV']
['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG', 'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAG

KeyError: 'CD'

In [137]:
len(l[3])

219

In [4]:
from Bio import PDB

def extract_chain_info(pdb_filepath, chain_id):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_filepath)

    chain = None
    for model in structure:
        for c in model:
            if c.id == chain_id:
                chain = c
                break
    
    if chain is None:
        raise ValueError(f"Chain '{chain_id}' not found in the structure.")

    chain_info = []
    for residue in chain:
        residue_id = residue.id[1]
        residue_name = residue.resname
        chain_info.append((residue_id, residue_name))
    
    return chain_info

pdb_filepath = path_protein
chain_id = "A"  # Specify the chain ID you want to extract
chain_info = extract_chain_info(pdb_filepath, chain_id)

for residue_id, residue_name in chain_info:
    print(f"Residue ID: {residue_id}, Residue Name: {residue_name}")


Residue ID: 1, Residue Name: GLU
Residue ID: 2, Residue Name: LEU
Residue ID: 3, Residue Name: VAL
Residue ID: 4, Residue Name: MET
Residue ID: 5, Residue Name: THR
Residue ID: 6, Residue Name: GLN
Residue ID: 7, Residue Name: THR
Residue ID: 8, Residue Name: PRO
Residue ID: 9, Residue Name: LEU
Residue ID: 10, Residue Name: SER
Residue ID: 11, Residue Name: LEU
Residue ID: 12, Residue Name: PRO
Residue ID: 13, Residue Name: VAL
Residue ID: 14, Residue Name: SER
Residue ID: 15, Residue Name: LEU
Residue ID: 16, Residue Name: GLY
Residue ID: 17, Residue Name: ASP
Residue ID: 18, Residue Name: GLN
Residue ID: 19, Residue Name: ALA
Residue ID: 20, Residue Name: SER
Residue ID: 21, Residue Name: ILE
Residue ID: 22, Residue Name: SER
Residue ID: 23, Residue Name: CYS
Residue ID: 24, Residue Name: ARG
Residue ID: 25, Residue Name: SER
Residue ID: 26, Residue Name: SER
Residue ID: 27, Residue Name: GLN
Residue ID: 1027, Residue Name: SER
Residue ID: 2027, Residue Name: LEU
Residue ID: 3027, R

In [ ]:
from Bio import PDB

def find_original_residue(pocket_filepath, protein_filepath, threshold=0.5):
    parser = PDB.PDBParser(QUIET=True)
    
    # Parse pocket structure
    pocket_structure = parser.get_structure("pocket", pocket_filepath)
    pocket_atoms = [atom for atom in pocket_structure.get_atoms()]
    
    # Parse full protein structure
    protein_structure = parser.get_structure("protein", protein_filepath)
    
    for pocket_atom in pocket_atoms:
        pocket_coord = pocket_atom.get_coord()
        
        for model in protein_structure:
            for chain in model:
                for residue in chain:
                    for atom in residue:
                        distance = atom - pocket_coord
                        if distance <= threshold:
                            return residue
    return None

pocket_filepath = "path_to_pocket.pdb"
protein_filepath = "path_to_protein.pdb"
threshold = 1.0  # You can adjust the threshold distance as needed

residue_with_atom = find_original_residue(pocket_filepath, protein_filepath, threshold)

if residue_with_atom:
    print("Atom in pocket corresponds to:")
    print("Chain:", residue_with_atom.parent.id)
    print("Residue ID:", residue_with_atom.id[1])
else:
    print("No corresponding residue found.")


## Get atoms from Molecule

In [20]:
molecule = Chem.MolFromPDBFile(path_protein,flavor=2,sanitize=True, removeHs=True)
print(molecule.GetNumAtoms())
print(molecule.GetNumBonds())

6859
6813


In [21]:
molecule = Chem.MolFromPDBFile(path_pocket, flavor=2,sanitize=True, removeHs=True)

In [76]:
atom = molecule.GetAtoms()[1]
atom_res = atom.GetPDBResidueInfo()
conformer = molecule.GetConformer(0)
coords = conformer.GetPositions()
print(coords[1])
print(molecule.GetNumAtoms(), atom.GetSymbol(), atom.GetChiralTag())
print(atom_res.GetResidueNumber(), atom_res.GetResidueName(), atom_res.GetChainId(),
      atom_res.GetInsertionCode(), atom_res.GetOccupancy(), atom_res.GetTempFactor())

[28.614 18.875 -6.927]
371 C CHI_TETRAHEDRAL_CCW
4027 HIS     1.0 27.1


In [30]:
for atom in molecule.GetAtoms():
    atom_res = atom.GetPDBResidueInfo()
    print(atom_res.GetResidueNumber(), atom_res.GetResidueName(), atom_res.GetChainId(),
          atom_res.GetInsertionCode())

4027 HIS    
4027 HIS    
4027 HIS    
4027 HIS    
4027 HIS    
4027 HIS    
4027 HIS    
4027 HIS    
4027 HIS    
4027 HIS    
28 ASN    
28 ASN    
28 ASN    
28 ASN    
28 ASN    
28 ASN    
28 ASN    
28 ASN    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
32 TYR    
34 HIS    
34 HIS    
34 HIS    
34 HIS    
34 HIS    
34 HIS    
34 HIS    
34 HIS    
34 HIS    
34 HIS    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
36 TYR    
90 GLN    
90 GLN    
90 GLN    
90 GLN    
90 GLN    
90 GLN    
90 GLN    
90 GLN    
90 GLN    
91 VAL    
91 VAL    
91 VAL    
91 VAL    
91 VAL    
91 VAL    
91 VAL    
92 THR    
92 THR    
92 THR    
92 THR    
92 THR    
92 THR    
92 THR    
93 HIS    
93 HIS    
93 HIS    
93 HIS    
93 HIS    
93 HIS    
93 HIS    
93 HIS    
93 HIS    
93 HIS    
94 VAL    
94 VAL    
94 VAL    
94 VAL    
9

## Testing the esm model

In [31]:
# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (

In [32]:
def pdb_to_sequences(pdb_filepath: str) -> List[str]:
    pdb_parser = PDB.PDBParser(QUIET=True)
    structure = pdb_parser.get_structure("X", pdb_filepath)
    # Using C-N
    ppb = PDB.PPBuilder()
    polypeptide_sequences = [
        str(pp.get_sequence()) for pp in ppb.build_peptides(structure)
    ]
    return polypeptide_sequences

In [48]:
def parse_pdb_files(folder_path: str):
    """Parse data from folders with the following structure:
    pdb/
    ├─ protein_1.pdb
    ├─ protein_2.pdb

    Where, for example `protein_1.pdb` contains a protein.
    Returns a list with the protein filename and the protein sequence.
    """
    all_pdb_files = list(pathlib.Path(folder_path).glob("*.pdb"))
    data = []
    for file in all_pdb_files:
        protein = pdb_to_sequences(file)[0]
        filename = str(file).replace(".pdb", "")
        data.append((filename, protein))
    return data

In [84]:
lista = parse_pdb_files("../../dataset2016/1a4k/")
lista = lista[1:]

In [85]:
lista

[('../../dataset2016/1a4k/1a4k_protein',
  'ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG')]

In [86]:
len(lista[0][1])

217

In [87]:
batch_labels, batch_strs, _ = batch_converter(lista)
print(batch_labels, batch_strs)

['../../dataset2016/1a4k/1a4k_protein'] ['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG']


In [88]:
dataset = FastaBatchedDataset(batch_labels, batch_strs)

In [92]:
batches = dataset.get_batch_indices(1, extra_toks_per_seq=1)
data_loader = torch.utils.data.DataLoader(dataset,collate_fn=batch_converter,batch_sampler=batches)

In [66]:
for _, (labels, strs, toks) in enumerate(tqdm(data_loader)):
    print(labels, strs, toks)

100%|██████████| 4/4 [00:00<00:00, 812.61it/s]

[2] ['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR'] tensor([[ 0,  9,  4,  7, 20, 11, 16, 11, 14,  4,  8,  4, 14,  7,  8,  4,  6, 13,
         16,  5,  8, 12,  8, 23, 10,  8,  8, 16,  8,  4,  4, 21,  8, 17,  6, 17,
         11, 19,  4, 21, 22, 19,  4, 16, 15, 14,  6, 16,  8, 14, 15,  4,  4, 12,
         19, 15,  7,  8, 17, 10, 18,  8,  6,  7, 14, 13, 10, 18,  8,  6,  8,  6,
          8,  6, 11, 13, 18, 11,  4, 15, 12,  8, 10,  7,  9,  5,  9, 13,  4,  6,
          7, 19, 18, 23,  8, 16,  7, 11, 21,  7, 14, 14, 11, 18,  6,  6,  6, 11,
         15,  4,  9, 12, 15, 10, 11,  7,  5,  5, 14,  8,  7, 18, 12, 18, 14, 14,
          8, 13,  9, 16,  4, 15,  8,  6, 11,  5,  8,  7,  7, 23,  4,  4, 17, 17,
         18, 19, 14, 10,  9,  5, 15,  7, 16, 22, 15,  7, 13, 17,  5,  4, 16,  8,
          6, 17,  8, 16,  9,  8,  7, 11,  9, 1

In [67]:
repr_layers = [(i + model.num_layers + 1) % (model.num_layers + 1) for i in [8]]

In [68]:
repr_layers

[1]

In [69]:
with torch.no_grad():
    for _, (labels, strs, toks) in enumerate(tqdm(data_loader)):
        out = model(toks, repr_layers=repr_layers, return_contacts=False)
        _ = out['logits'].to(device='cpu')

        representations = {
            layer: t.to(device='cpu')
            for layer, t in out['representations'].items()
        }
        # print(representations)
        for i, label in enumerate(labels):
            result = {'label': label}
            truncate_len = min(2000, len(strs[i]))

            
            result['per_tok'] = {
                layer: t[i, 1:truncate_len + 1].clone()
                for layer, t in representations.items()
                }
            
            result['mean'] = {
                layer:
                    t[i,
                        1:truncate_len + 1].mean(0).clone().unsqueeze(0)
                for layer, t in representations.items()
            }
            
            result['bos'] = {
                layer: t[i, 0].clone()
                for layer, t in representations.items()
            }

100%|██████████| 4/4 [00:00<00:00, 48.25it/s]


In [59]:
result['per_tok'][1].size()

torch.Size([217, 320])

In [60]:
result['mean'][33].size()

KeyError: 33

In [ ]:
result['mean']

{33: tensor([[ 0.0133, -0.0756, -0.0588,  ..., -0.0962, -0.0233,  0.1151]])}

In [ ]:
result['bos']

{33: tensor([ 0.1019,  0.0491,  0.0177,  ..., -0.2179,  0.2239,  0.0176])}

In [76]:
representations[1]

tensor([[[-0.1915,  0.5826, -0.0663,  ...,  0.3430, -0.1762, -4.3831],
         [-0.2651, -0.0640,  0.1722,  ..., -0.1987,  0.3459, -3.3668],
         [ 0.0815,  0.1780,  0.4591,  ...,  0.3362,  0.3494, -3.1021],
         ...,
         [-0.1838, -0.5669, -0.0776,  ..., -0.1460, -0.0283, -3.4215],
         [-0.2369, -0.3772,  0.2040,  ..., -0.2198, -0.2494, -3.2558],
         [ 0.8461, -0.3517, -0.0523,  ..., -0.1723, -0.8778, -3.2876]]])

## adequate to many chains, instead of many elements in the dataset

In [82]:
def pdb_to_sequences(pdb_filepath: str) -> List[str]:
    pdb_parser = PDB.PDBParser(QUIET=True)
    structure = pdb_parser.get_structure("X", pdb_filepath)
    # Using C-N
    ppb = PDB.PPBuilder()
    polypeptide_sequences = [
        str(pp.get_sequence()) for pp in ppb.build_peptides(structure)
    ]
    return polypeptide_sequences

In [83]:
def parse_pdb_files2(pdb_filepath: str) -> List[str]:
    data = []
    protein = pdb_to_sequences(pdb_filepath)
    for i, chain in enumerate(protein):
        data.append((i, chain))
    return data

In [100]:
lista = parse_pdb_files2(path_protein)

In [101]:
lista

[(0,
  'ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG'),
 (1,
  'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKV'),
 (2,
  'ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR'),
 (3,
  'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP')]

In [102]:
batch_labels, batch_strs, _ = batch_converter(lista)

In [127]:
batch_labels, batch_strs

([0, 1, 2, 3],
 ['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG',
  'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKV',
  'ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR',
  'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP'])

In [143]:
dataset = FastaBatchedDataset(batch_labels, batch_strs)

In [117]:
dataset.__getitem__(3)

(3,
 'QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP')

In [144]:
batches = dataset.get_batch_indices(1, extra_toks_per_seq=1)
data_loader = torch.utils.data.DataLoader(dataset,collate_fn=batch_converter,batch_sampler=batches)

In [154]:
results = dict()
with torch.no_grad():
    for i, (labels, strs, toks) in enumerate(data_loader):
        print(i, labels, strs)
        out = model(toks, repr_layers=repr_layers, return_contacts=False)
        results[labels[0]] = out['representations'][1]

0 [2] ['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR']
1 [0] ['ELVMTQTPLSLPVSLGDQASISCRSSQSLLHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQVTHVPPTFGGGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNRG']
2 [1] ['QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKV']
3 [3] ['QVQLLESGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMGWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCVQAERLRRTFDYWGAGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEP']


In [155]:
results

{2: tensor([[[-0.0775,  0.5596, -0.0402,  ...,  0.2912, -0.1636, -4.1395],
          [-0.3117, -0.1180,  0.3821,  ..., -0.0972,  0.4499, -2.9818],
          [-0.1690, -0.0445,  0.0805,  ...,  0.1739,  0.2628, -2.6350],
          ...,
          [-0.3568,  0.4562, -0.0437,  ..., -0.0302,  0.1226, -3.9014],
          [-0.1342,  0.1431, -0.2939,  ...,  0.4557, -0.0637, -3.6827],
          [ 0.7119,  0.1444,  0.0844,  ..., -0.1247, -0.6918, -3.5571]]]),
 0: tensor([[[-0.0821,  0.5629, -0.0410,  ...,  0.2861, -0.1638, -4.1420],
          [-0.3090, -0.1121,  0.3785,  ..., -0.0972,  0.4500, -2.9836],
          [-0.1675, -0.0396,  0.0763,  ...,  0.1776,  0.2635, -2.6344],
          ...,
          [-0.2463, -0.0132, -0.1701,  ...,  0.5264, -0.0540, -3.6004],
          [-0.3135, -0.0701, -0.2101,  ...,  0.0784, -0.1260, -3.7980],
          [ 0.7204,  0.1123,  0.0805,  ..., -0.0528, -0.8757, -3.6267]]]),
 1: tensor([[[-0.1908,  0.5888, -0.0579,  ...,  0.3373, -0.1763, -4.3784],
          [-0.2637,